In [132]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, Input
from keras import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Model Architecture

In [15]:
input_shape = Input((2,10)) # (time_step, embedding_lenght)
output = SimpleRNN(3, return_sequences=True)(input_shape)
model = Model(inputs=input_shape, outputs=output)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 2, 10)             0         
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 2, 3)              42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# Example1

In [31]:
text="나랑 점심 먹으러 갈래 메뉴는 햄버거 점심 메뉴 좋지"

In [34]:
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
# [0]을 해주지 않으면 [[contents]]와 같은 리스트 안의 리스트 형태로 저장 됨.
# [0]을 해주면 [contents]와 같은 하나의 리스트로 저장됨.

In [36]:
encoded

[2, 1, 3, 4, 5, 6, 1, 7, 8]

In [37]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 9


In [38]:
print(t.word_index)

{'점심': 1, '나랑': 2, '먹으러': 3, '갈래': 4, '메뉴는': 5, '햄버거': 6, '메뉴': 7, '좋지': 8}


In [39]:
sequences = list()
for c in range(1, len(encoded)):
    sequence = encoded[c-1:c+1] # 단어를 두개씩 묶어서 저장해준다. 이는 X와 Y의 관계를 구성하기 위함이다.
    sequences.append(sequence)
print('단어 묶음의 개수: %d' % len(sequences))

단어 묶음의 개수: 8


아래와 같이 단순히 단어에 index를 통해서 학습데이터로 전처리할 수 있다.

In [43]:
sequences
# 위의 결과는 아래와 같음. 첫번째 열이 X가 되고, 두번째 열이 예측해야할 다음 단어인 Y가 될 것.
# [나랑, 점심],
# [점심, 먹으러],
# [먹으러, 갈래],
# [갈래, 메뉴는],
# [메뉴는, 햄버거]
# [햄버거, 점심]
# [점심, 메뉴]
# [메뉴, 좋지]

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 1], [1, 7], [7, 8]]

In [46]:
X, y = zip(*sequences) # 첫번째 열이 X, 두번째 열이 y가 됨.
X=np.array(X) # 타입을 배열로 변환
y=np.array(y) # 타입을 배열로 변환

In [49]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size) # 원 핫 인코딩
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [50]:
model = Sequential()
model.add(Embedding(vocab_size, 9, input_length=1))
# 단어 집합의 크기는 9. 임베딩 벡터의 크기는 9. 각 sample의 길이는 단어 한 개이므로 길이는 1.
model.add(SimpleRNN(9))
# RNN의 결과값으로 나오는 벡터의 차원 또한 9로 한다. 더 크게 해주어도 상관은 없음.
model.add(Dense(vocab_size, activation='softmax'))
# 출력층을 지나서 나오는 벡터의 크기도 9로 한다.

In [52]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 9s - loss: 2.2128 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 2.2092 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 2.2057 - acc: 0.0000e+00
Epoch 4/500
 - 0s - loss: 2.2022 - acc: 0.0000e+00
Epoch 5/500
 - 0s - loss: 2.1988 - acc: 0.0000e+00
Epoch 6/500
 - 0s - loss: 2.1953 - acc: 0.0000e+00
Epoch 7/500
 - 0s - loss: 2.1918 - acc: 0.0000e+00
Epoch 8/500
 - 0s - loss: 2.1883 - acc: 0.0000e+00
Epoch 9/500
 - 0s - loss: 2.1849 - acc: 0.1250
Epoch 10/500
 - 0s - loss: 2.1814 - acc: 0.2500
Epoch 11/500
 - 0s - loss: 2.1779 - acc: 0.2500
Epoch 12/500
 - 0s - loss: 2.1745 - acc: 0.2500
Epoch 13/500
 - 0s - loss: 2.1710 - acc: 0.2500
Epoch 14/500
 - 0s - loss: 2.1675 - acc: 0.2500
Epoch 15/500
 - 0s - loss: 2.1640 - acc: 0.3750
Epoch 16/500
 - 0s - loss: 2.1605 - acc: 0.3750
Epoch 17/500
 - 0s - loss: 2.1570 - acc: 0.3750
Epoch 18/500
 - 0s - loss: 2.1535 - acc: 0.3750
Epoch 19/500
 - 0s - loss: 2.1499 - acc: 0.3750
Epoch 20/500
 - 0

In [55]:
t.word_index.items()

dict_items([('점심', 1), ('나랑', 2), ('먹으러', 3), ('갈래', 4), ('메뉴는', 5), ('햄버거', 6), ('메뉴', 7), ('좋지', 8)])

In [69]:
def predict_next_word(model, t, current_word): # 모델, 토크나이저, 현재 단어를 받아온다.
    encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
    encoded = np.array(encoded) # 현재 단어에 대한 정수 인코딩
    result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
    for word, index in t.word_index.items(): # 위에서 실습한 것처럼 단어와 인덱스를 리턴.
        if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            return word # 그 단어를 출력

In [61]:
encoded = t.texts_to_sequences(['먹으러'])[0]
encoded

[3]

In [62]:
encoded = np.array(encoded)
encoded

array([3])

In [63]:
model.predict_classes(encoded, verbose=0)

array([4], dtype=int64)

In [70]:
print(predict_next_word(model, t, '먹으러'))

갈래


아래 sentence_generation 함수는 예측단어를 다시 입력단어로 바꾸어 계속해서 뒤의 단어를 생성하는 함수이다.

In [72]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = np.array(encoded) # 현재 단어에 대한 정수 인코딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = word # 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [73]:
print(sentence_generation(model, t, '먹으러', 6))

먹으러 갈래 메뉴는 햄버거 점심 먹으러 갈래


# Example2
위에서는 '점심' 뒤에 '먹으러'와 '메뉴'와 같이 한 단어 뒤에 등장단어가 다르다보니 정확도가 다소 떨어지는 점이 있다. 이런 문제를 해결하기 위해서 문맥을 중심으로 학습하는 방법을 아래 예시로 실습해본다.

time step|X|y
----|----|----
1|경마장에|있는
2|경마장에 있는|말이
3|경마장에 있는 말이|뛰고
4|경마장에 있는 말이 뛰고|있다
5|그의|말이
6|그의 말이|법이다
7|가는|말이
8|가는 말이|고와야
9|가는 말이 고와야|오는
10|가는 말이 고와야 오는|말이
11|가는 말이 고와야 오는 말이|곱다

In [76]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [78]:
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [80]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [81]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [84]:
sequences = list()
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('훈련 데이터의 개수: %d' % len(sequences))

훈련 데이터의 개수: 11


In [85]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

아래는 위의 index를 word로 변환한 결과이다.

In [122]:
word_index = np.array(list(t.word_index.items()))
for indices in sequences:
    indices = np.array(indices) - 1
    words = word_index[indices][:,0]
    print(' '.join(words))

경마장에 있는
경마장에 있는 말이
경마장에 있는 말이 뛰고
경마장에 있는 말이 뛰고 있다
그의 말이
그의 말이 법이다
가는 말이
가는 말이 고와야
가는 말이 고와야 오는
가는 말이 고와야 오는 말이
가는 말이 고와야 오는 말이 곱다


모든 데이터에서 가장 길이가 긴 문장을 기준으로 padding한다.

In [123]:
print(max(len(l) for l in sequences))

6


In [125]:
sequences = pad_sequences(sequences, maxlen=6, padding='pre')

In [126]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

target은 마지막 열로 정한다.

In [129]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [130]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [135]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]])

In [139]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=5))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 2.4590 - acc: 0.0909
Epoch 2/200
 - 0s - loss: 2.4460 - acc: 0.1818
Epoch 3/200
 - 0s - loss: 2.4327 - acc: 0.1818
Epoch 4/200
 - 0s - loss: 2.4189 - acc: 0.2727
Epoch 5/200
 - 0s - loss: 2.4048 - acc: 0.4545
Epoch 6/200
 - 0s - loss: 2.3902 - acc: 0.4545
Epoch 7/200
 - 0s - loss: 2.3751 - acc: 0.4545
Epoch 8/200
 - 0s - loss: 2.3594 - acc: 0.3636
Epoch 9/200
 - 0s - loss: 2.3431 - acc: 0.3636
Epoch 10/200
 - 0s - loss: 2.3262 - acc: 0.3636
Epoch 11/200
 - 0s - loss: 2.3087 - acc: 0.3636
Epoch 12/200
 - 0s - loss: 2.2906 - acc: 0.4545
Epoch 13/200
 - 0s - loss: 2.2718 - acc: 0.4545
Epoch 14/200
 - 0s - loss: 2.2525 - acc: 0.4545
Epoch 15/200
 - 0s - loss: 2.2326 - acc: 0.4545
Epoch 16/200
 - 0s - loss: 2.2121 - acc: 0.4545
Epoch 17/200
 - 0s - loss: 2.1913 - acc: 0.4545
Epoch 18/200
 - 0s - loss: 2.1700 - acc: 0.4545
Epoch 19/200
 - 0s - loss: 2.1485 - acc: 0.4545
Epoch 20/200
 - 0s - loss: 2.1269 - acc: 0.3636
Epoch 21/200
 - 0s - loss: 2.1052 - acc: 0.3636
E

In [141]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [142]:
print(sentence_generation(model,t,'경마장에',4))

경마장에 있는 말이 뛰고 있다


In [145]:
print(sentence_generation(model,t,'그의',2))

그의 말이 법이다


In [147]:
print(sentence_generation(model,t,'가는',5))

가는 말이 고와야 오는 말이 곱다
